In [1]:
!nvidia-smi

Thu Nov 25 06:57:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.67       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:65:00.0 Off |                  N/A |
| 60%   52C    P5    87W / 370W |   1581MiB / 24265MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:B3:00.0 Off |                  N/A |
| 76%   

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform as st
import os
import gc
import warnings
from sklearn.metrics import classification_report, average_precision_score
from Parse_TFrecords import *
from define_model import *
from load_data import *
from utilities import *
 
print(tf.__version__)

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

gpus = tf.config.list_physical_devices(device_type='GPU')
tf.config.set_visible_devices(devices=gpus[0], device_type='GPU')

2.5.0


# Pretrain model

In [3]:
files = 'copd_no_label.tfrecords'

BATCH_SIZE = 32

train_dataset = (tf.data.TFRecordDataset(
    files, compression_type=None, buffer_size=BATCH_SIZE*10, num_parallel_reads=32)
.map(parse_TFrecord_pretrain)
.batch(BATCH_SIZE, drop_remainder=True)
.shuffle(256))

In [4]:
checkpoint_filepath = 'checkpoints/AUC/checkpoint_pretrain_Dnet121'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True,
    monitor='auc',
    mode='max')

callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.EarlyStopping(mode='max', patience=3, monitor='auc'),
            model_checkpoint_callback]

metric = tf.keras.metrics.AUC()

model = define_model_pretrain(archi='Dnet121')

model.summary()

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=metric)

model.fit(train_dataset, epochs=5, shuffle=True, callbacks=callback)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

Epoch 1/5
12/12 [==============================] - 12s 124ms/step - loss: 0.9071 - auc: 0.7115
Epoch 2/5
12/12 [==============================] - 2s 125ms/step - loss: 0.4365 - auc: 0.8642
Epoch 3/5
12/12 [==============================] - 2s 125ms/step - loss: 0.2552 - auc: 0.9372
Epoch 4/5
12/12 [==============================] - 2s 126ms/step - loss: 0.1516 - auc: 0.9748


KeyboardInterrupt: 

# Transfer learning

In [3]:
pos = 27297
neg = 119133

total = pos+neg

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.61
Weight for class 1: 2.68


In [4]:
BATCH_SIZE = 32

record_file_train = 'copd_train_new.tfrecords'
train_dataset = (tf.data.TFRecordDataset(
    record_file_train, buffer_size=BATCH_SIZE*10, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_train)
.shuffle(total)
.batch(BATCH_SIZE))

record_file_val = 'copd_val_new.tfrecords'
val_dataset = (tf.data.TFRecordDataset(
    record_file_val, buffer_size=BATCH_SIZE*10, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_train)
.shuffle(36601)
.batch(BATCH_SIZE))

record_file_test = 'copd_test_new.tfrecords'
test_dataset = (tf.data.TFRecordDataset(
    record_file_test, buffer_size=BATCH_SIZE*10, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_test)
.batch(BATCH_SIZE))

y_train = get_data_label('train')
y_val = get_data_label('val')
y_test = get_data_label('test')

## Different loss functions

In [5]:
archi='Dnet121'

checkpoint_filepath = 'checkpoints/AUC/checkpoint_KLDiv_Dnet121'
monitor_ = 'val_auc'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True,
    monitor=monitor_,
    mode='max')
    
callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.EarlyStopping(mode='max', patience=3, monitor=monitor_),
            model_checkpoint_callback]

model = load_model_from_pretrain(archi)

# loss_func = binary_focal_loss
# loss_func = tf.keras.losses.CategoricalHinge()
loss_func = tf.keras.losses.KLDivergence()
# loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

metric = tf.keras.metrics.AUC(name='auc')

model.compile(loss=loss_func,
                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=metric)

model.fit(train_dataset, epochs=20, shuffle=True, validation_data=val_dataset, callbacks=callback, class_weight=class_weight)

y_preds = model.predict(test_dataset)

test_CI(y_preds, y_test)
    
thresh = get_thresh(y_test, y_preds, 'Youden')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

thresh = get_thresh(y_test, y_preds, 'G-mean')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

thresh = 0.5
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

gc.collect()

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/20
4576/4576 [==============================] - 440s 91ms/step - loss: 3.9576e-04 - auc: 0.5001 - val_loss: 0.0118 - val_auc: 0.5029
Epoch 2/20
4576/4576 [==============================] - 431s 92ms/step - loss: -7.9117e-07 - auc: 0.5000 - val_loss: 0.0135 - val_auc: 0.5031
Epoch 3/20
4576/4576 [==============================] - 443s 93ms/step - loss: -8.0498e-07 - auc: 0.5000 - val_loss: 0.0155 - val_auc: 0.5035
Epoch 4/20
4576/4576 [==============================] - 445s 94ms/step - loss: -8.0568e-07 - auc: 0.5000 - val_loss: 0.0131 - val_auc: 0.5027
Epoch 5/20
4576/4576 [==============================] - 444s 94ms/step - loss: -8.0583e-07 - auc: 0.5000 - val_loss: 0.0117 - val_auc: 0.5021
Epoch 6/20
4576/4576 [==============================] - 442s 93ms/step - loss: -8.0589e-07 - auc: 0.5000 - val_loss: 0.0155 - val_auc: 0.5032
Sample auc 

192

In [6]:
archi='Dnet121'

checkpoint_filepath = 'checkpoints/AUC/checkpoint_Hinge_Dnet121'
monitor_ = 'val_auc'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True,
    monitor=monitor_,
    mode='max')
    
callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.EarlyStopping(mode='max', patience=3, monitor=monitor_),
            model_checkpoint_callback]

model = load_model_from_pretrain(archi)

# loss_func = binary_focal_loss
loss_func = tf.keras.losses.CategoricalHinge()
# loss_func = tf.keras.losses.KLDivergence()

metric = tf.keras.metrics.AUC(name='auc')

model.compile(loss=loss_func,
                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=metric)

model.fit(train_dataset, epochs=20, shuffle=True, validation_data=val_dataset, callbacks=callback, class_weight=class_weight)

y_preds = model.predict(test_dataset)

test_CI(y_preds, y_test)
    
thresh = get_thresh(y_test, y_preds, 'Youden')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

thresh = get_thresh(y_test, y_preds, 'G-mean')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

thresh = 0.5
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

gc.collect()

Epoch 1/20


4576/4576 [==============================] - 432s 91ms/step - loss: 0.8728 - auc: 0.6483 - val_loss: 1.2505 - val_auc: 0.6345
Epoch 2/20
4576/4576 [==============================] - 420s 90ms/step - loss: 0.8589 - auc: 0.6532 - val_loss: 1.2270 - val_auc: 0.6498
Epoch 3/20
4576/4576 [==============================] - 419s 90ms/step - loss: 0.8553 - auc: 0.6560 - val_loss: 1.0837 - val_auc: 0.6288
Epoch 4/20
4576/4576 [==============================] - 418s 90ms/step - loss: 0.8482 - auc: 0.6625 - val_loss: 1.2034 - val_auc: 0.6598
Epoch 5/20
4576/4576 [==============================] - 419s 90ms/step - loss: 0.8448 - auc: 0.6662 - val_loss: 1.2570 - val_auc: 0.6566
Epoch 6/20
4576/4576 [==============================] - 420s 90ms/step - loss: 0.8425 - auc: 0.6666 - val_loss: 1.1058 - val_auc: 0.6420
Epoch 7/20
4576/4576 [==============================] - 419s 90ms/step - loss: 0.8373 - auc: 0.6713 - val_loss: 1.1332 - val_auc: 0.6621
Epoch 8/20
4576/4576 [==============================

169

In [7]:
archi='Dnet121'

checkpoint_filepath = 'checkpoints/AUC/checkpoint_Bfocal_Dnet121'
monitor_ = 'val_auc'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True,
    monitor=monitor_,
    mode='max')
    
callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.EarlyStopping(mode='max', patience=3, monitor=monitor_),
            model_checkpoint_callback]

model = load_model_from_pretrain(archi)

loss_func = binary_focal_loss
# loss_func = tf.keras.losses.CategoricalHinge()
# loss_func = tf.keras.losses.KLDivergence()

metric = tf.keras.metrics.AUC(name='auc')

model.compile(loss=loss_func,
                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=metric)

model.fit(train_dataset, epochs=20, shuffle=True, validation_data=val_dataset, callbacks=callback, class_weight=class_weight)

y_preds = model.predict(test_dataset)

test_CI(y_preds, y_test)
        
thresh = get_thresh(y_test, y_preds, 'Youden')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

thresh = get_thresh(y_test, y_preds, 'G-mean')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

thresh = 0.5
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

gc.collect()

Epoch 1/20


4576/4576 [==============================] - 427s 90ms/step - loss: 0.0114 - auc: 0.4999 - val_loss: 0.0039 - val_auc: 0.5000
Epoch 2/20
4576/4576 [==============================] - 420s 90ms/step - loss: 0.0113 - auc: 0.5000 - val_loss: 0.0039 - val_auc: 0.5000
Epoch 3/20
4576/4576 [==============================] - 419s 90ms/step - loss: 0.0113 - auc: 0.5000 - val_loss: 0.0039 - val_auc: 0.5000
Epoch 4/20
4576/4576 [==============================] - 418s 90ms/step - loss: 0.0113 - auc: 0.5000 - val_loss: 0.0039 - val_auc: 0.5000
Sample auc mean: 0.50
Samole auc std: 0.00
Sample auc CI: 0.00
Confidence interval for AUC: [0.50 - 0.50]
thresh: 1.0
              precision    recall  f1-score   support

           0       0.83      0.00      0.01     36808
           1       0.20      1.00      0.33      9026

    accuracy                           0.20     45834
   macro avg       0.51      0.50      0.17     45834
weighted avg       0.71      0.20      0.07     45834

0.1970506338569765

177

In [10]:
checkpoint_filepath = 'checkpoints/AUC/checkpoint_BCE_Dnet121'
archi = 'Dnet121'
model = define_model(archi)
model.load_weights(checkpoint_filepath)

In [11]:
y_preds = model.predict(test_dataset)

test_CI(y_preds, y_test)
        
thresh = get_thresh(y_test, y_preds, 'Youden')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

thresh = get_thresh(y_test, y_preds, 'G-mean')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

thresh = 0.5
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

gc.collect()

Sample auc mean: 0.74
Samole auc std: 0.00
Sample auc CI: 0.01
Confidence interval for AUC: [0.73 - 0.75]
thresh: 0.62232316
              precision    recall  f1-score   support

           0       0.89      0.68      0.77     36808
           1       0.34      0.67      0.45      9026

    accuracy                           0.68     45834
   macro avg       0.62      0.67      0.61     45834
weighted avg       0.78      0.68      0.71     45834

0.4129518843544724
thresh: 0.61584055
              precision    recall  f1-score   support

           0       0.89      0.67      0.77     36808
           1       0.34      0.68      0.45      9026

    accuracy                           0.67     45834
   macro avg       0.62      0.67      0.61     45834
weighted avg       0.78      0.67      0.70     45834

0.4129518843544724
thresh: 0.5
              precision    recall  f1-score   support

           0       0.91      0.53      0.67     36808
           1       0.29      0.80      0.43

954